In [85]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize  

In [9]:
#Debugging
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [2]:
df_link = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df = pd.read_csv(df_link)
df.drop(columns=['Lat', 'Long','Province/State'], inplace=True)

In [3]:
result = df.groupby('Country/Region').sum()
result.reset_index(inplace=True)
# Rename 
result["Country/Region"]= result["Country/Region"].replace("Mainland China", "China")
result["Country/Region"]= result["Country/Region"].replace("Others", "Diamond Princess")

In [123]:
def LocationRename(location):
    return {
        'North Macedonia' : 'Macedonia',
        'South Korea' : 'Korea (Republic of)',
        'Vatican City' : 'Vatican'
    }.get(location,location)
def GetJson(location):
    location_api = LocationRename(location)
    api_link_base = 'https://restcountries.eu/rest/v2/name/'
    apt_link = api_link_base + location_api
    response = requests.get(apt_link)
    if response.status_code == 200:
        return ({
            "name" : location,
            "flag" : response.json()[0]['flag'],
            "population" : response.json()[0]['population'],
            "region" : response.json()[0]['region']
        })
    else:
        if location == 'Diamond Princess':
            return ({
                "name" : location,
                "flag" : "https://cdn.iconscout.com/icon/free/png-256/ship-pessanger-boat-transportation-vehicle-emoj-symbol-30767.png",
                "population" : 3770,
                "region" : "Asia"
            })
        return ({
            "name" : location,
            "flag" : "",
            "population" : 0,
            "region" : ""
        })

In [124]:
country_json = json.loads("{\"country\" :[]}")
for country in result["Country/Region"]:
    country_json['country'].append(GetJson(country))

In [125]:
pd.read_json(json.dumps(country_json['country'])).to_csv('temp.csv')

In [126]:
df = pd.read_csv('temp.csv')
df.drop(columns=['Unnamed: 0'],inplace=True)